In [29]:
def Gradient(input, variable):
    gradient = ''
    split_1 = input.split("+")
    split_2 = []
    for i in split_1:
        split_2.append(i.split("*"))
    for i in split_2:
        if variable in i:
            i.remove(variable)
            if len(i) == 0:
                gradient+="*".join("1")

            else:   
                gradient+="*".join(i)

            gradient = gradient + "+"

    if gradient[-1] == "+":
        return gradient[:-1]
    else:
        return gradient
    

def Simplyfi(expression):
        if len(expression)>1:
            split_1 = expression.split("+")
            split_2 = []
            for i in split_1:
                split_2.append(i.split("*"))

        result = ""

        for i in range(len(split_2)):
            if all(expr == '1' for expr in split_2[i]):
                return '1'
            split2_holder = [k for k in split_2[i] if k != '1']
            split_2 = split2_holder
            for j in range(len(split_2)):
                if len(split_2) == 1:
                    result += split_2[j]
                    break
                result += split_2[j]+"*"
            result += "+"

        while result[0] == "+" or result[0] == "*":
            result= result[1:]

        while result[-1] == "+" or result[-1] == "*":
            result= result[:-1]
        return result

In [30]:
class Node:
    instances = []
    def __init__(self, id, layer,operation, *input):
        self.id = id
        self.layer = layer
        self.input = input
        self.operation = operation
        self.gradients = {}
        Node.instances.append(self)
    def Calculate(self):
        if self.operation:
            self.output = self.operation(self.input)
        else:
            self.output = self.input

    def all_gradients(self):
        for i in self.input:
            self.gradients[i]=Gradient(self.output,i)


    def Find_Gradient(self, variable):
        def find_all_routes(current_node, target_layer, current_path, all_routes, initial_node):

            if current_node.layer == target_layer:
                all_routes.append(current_path)
                return
            for node in Node.instances:
                if current_node.layer+1 == node.layer:
                    if any(inpt == current_node.id for inpt in node.input):
                        current_path.append(node)
                        find_all_routes(node,target_layer,current_path,all_routes,initial_node)
                        current_path = [initial_node]

        initial_nodes = [node for node in Node.instances if variable in node.input and node.layer != 0]
        all_routes = []
        for initial_node in initial_nodes:
            find_all_routes(initial_node, self.layer, [initial_node], all_routes, initial_node)

        gradient_list = []
        for route in all_routes:
            gradient = ""
            for k, node in enumerate(route):
                if k == 0:
                    gradient += node.gradients[variable]
                else:
                    gradient += "*" + route[k].gradients[route[k - 1].id]

            gradient = Simplyfi(gradient)
            gradient_list.append(gradient)

        return gradient_list

In [31]:
def Add(*input):
    input = input[0]
    return "+".join(input)
def Multiply(*input):
    input = input[0]
    return "*".join(input)

In [32]:
a= Node("a",0,None,"a")
b = Node("b",0,None,"b")
c = Node("c",0,None,"c")
a.Calculate()
b.Calculate()
c.Calculate()

<bound method Node.Calculate of <__main__.Node object at 0x0000021EE7EBABD0>>

In [33]:
d = Node("d",1,Multiply,a.id,b.id)
e = Node("e",1,Multiply,a.id,b.id,c.id)
f = Node("f",1,Add,b.id,c.id)
d.Calculate()
e.Calculate()
f.Calculate()

In [34]:
g = Node("g",2,Multiply,d.id,e.id,f.id)
g.Calculate()

In [36]:
d.all_gradients()
e.all_gradients()
f.all_gradients()
g.all_gradients()

g.Find_Gradient("b")

['a*e*f', 'a*c*d*f', 'd*e']